ASSIGNMENT 3

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from scipy.stats import entropy


In [47]:
# Data preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])


In [49]:
# Load dataset
train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)


In [50]:
# Split the training dataset
X_train, X_validation, y_train, y_validation = train_test_split(train_dataset.data.numpy(),
                                                              train_dataset.targets.numpy(),
                                                              test_size=0.2, random_state=2020)


In [51]:
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1) / 255
X_validation = torch.tensor(X_validation, dtype=torch.float32).unsqueeze(1) / 255
y_train = torch.tensor(y_train, dtype=torch.long)
y_validation = torch.tensor(y_validation, dtype=torch.long)



In [52]:
X_test = torch.tensor(test_dataset.data.numpy(), dtype=torch.float32).unsqueeze(1) / 255
y_test = torch.tensor(test_dataset.targets.numpy(), dtype=torch.long)

In [53]:
# Create data loaders
def create_data_loader(X, y, batch_size):
    return DataLoader(TensorDataset(X, y), batch_size=batch_size, shuffle=True)


In [54]:
train_loader = create_data_loader(X_train, y_train, batch_size=512)
validation_loader = create_data_loader(X_validation, y_validation, batch_size=512)
test_loader = create_data_loader(X_test, y_test, batch_size=512)



In [55]:
class CNNModel(nn.Module):
    def __init__(self): # Corrected the method name to __init__
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [56]:
# Define training function
def train_model(model, train_loader, criterion, optimizer, device, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')




In [57]:
# Define evaluation function
def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total



In [58]:
# Active Learning - Uncertainty Metrics
def least_confidence(predictions):
    confidences, _ = predictions.max(dim=1)
    return confidences.argsort()



In [59]:
def prediction_entropy(predictions):
    probabilities = torch.softmax(predictions, dim=1)
    entropies = entropy(probabilities.cpu().numpy(), axis=1)
    return torch.tensor(entropies).argsort(descending=True)



In [60]:
def margin_sampling(predictions):
    probabilities = torch.softmax(predictions, dim=1)
    top2_probs, _ = probabilities.topk(2, dim=1)
    margins = top2_probs[:, 0] - top2_probs[:, 1]
    return margins.argsort()


In [61]:
# Active Learning - Diversity Metrics
def cosine_similarity_metric(features):
    similarities = cosine_similarity(features.cpu().numpy())
    diversities = 1 - similarities.sum(axis=1)
    return torch.tensor(diversities).argsort(descending=True)


In [62]:
def l2_norm_metric(features):
    distances = torch.cdist(features, features, p=2).sum(dim=1)
    return distances.argsort(descending=True)


In [63]:
def kl_divergence_metric(predictions):
    probabilities = torch.softmax(predictions, dim=1)
    mean_distribution = probabilities.mean(dim=0)
    kl_divs = entropy(probabilities.cpu().numpy().T, mean_distribution.cpu().numpy())
    return torch.tensor(kl_divs).argsort(descending=True)


In [72]:
# Train and evaluate without active learning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_without_al = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_without_al.parameters(), lr=0.001)

print("Training without Active Learning:")
train_model(model_without_al, train_loader, criterion, optimizer, device, epochs=30)
validation_accuracy_without_al = evaluate_model(model_without_al, validation_loader, device)
print(f'Validation Accuracy without Active Learning: {validation_accuracy_without_al}%')

Training without Active Learning:
Epoch 1/30, Loss: 0.9231703110197758
Epoch 2/30, Loss: 0.5347481259006135
Epoch 3/30, Loss: 0.45318579356721106
Epoch 4/30, Loss: 0.40938833324199025
Epoch 5/30, Loss: 0.37749939713072267
Epoch 6/30, Loss: 0.3543104405732865
Epoch 7/30, Loss: 0.33810041559503434
Epoch 8/30, Loss: 0.3222665615538333
Epoch 9/30, Loss: 0.3068042335675118
Epoch 10/30, Loss: 0.29680865321387634
Epoch 11/30, Loss: 0.2895380012849544
Epoch 12/30, Loss: 0.2801765360413714
Epoch 13/30, Loss: 0.27411900381458565
Epoch 14/30, Loss: 0.2668853982331905
Epoch 15/30, Loss: 0.25348374199994067
Epoch 16/30, Loss: 0.25181598913796405
Epoch 17/30, Loss: 0.2429173048189346
Epoch 18/30, Loss: 0.23320344042904834
Epoch 19/30, Loss: 0.22877632129065534
Epoch 20/30, Loss: 0.22708501270476808
Epoch 21/30, Loss: 0.21764474695033215
Epoch 22/30, Loss: 0.21279376206245829
Epoch 23/30, Loss: 0.2087788684888089
Epoch 24/30, Loss: 0.20324456802708038
Epoch 25/30, Loss: 0.1956401121743182
Epoch 26/30

In [66]:
# Active learning implementation
def active_learning_selection(model, loader, device, selection_method, n_samples=100):
    model.eval()
    all_indices = []
    all_scores = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            if selection_method in [least_confidence, prediction_entropy, margin_sampling]:
                scores = selection_method(outputs)
            else:
                features = inputs.view(inputs.size(0), -1)
                scores = selection_method(features)

            all_indices.extend(scores.cpu().numpy())
            all_scores.extend(scores.cpu().numpy())

    selected_indices = np.argsort(all_scores)[:n_samples]
    return selected_indices



In [67]:

# Select additional samples using active learning
selected_indices = active_learning_selection(model_without_al, validation_loader, device, least_confidence, n_samples=500)
X_selected = X_validation[selected_indices]
y_selected = y_validation[selected_indices]



In [68]:
# Combine selected samples with training data
X_train_al = torch.cat([X_train, X_selected])
y_train_al = torch.cat([y_train, y_selected])
train_loader_al = create_data_loader(X_train_al, y_train_al, batch_size=512)


In [69]:
# Train and evaluate with active learning
model_with_al = CNNModel().to(device)
optimizer_al = optim.Adam(model_with_al.parameters(), lr=0.001)


In [73]:
print("Training with Active Learning:")
train_model(model_with_al, train_loader_al, criterion, optimizer_al, device, epochs=30)
validation_accuracy_with_al = evaluate_model(model_with_al, validation_loader, device)
print(f'Validation Accuracy with Active Learning: {validation_accuracy_with_al}%')


Training with Active Learning:
Epoch 1/30, Loss: 0.1965095927840785
Epoch 2/30, Loss: 0.18978847657379352
Epoch 3/30, Loss: 0.18232213478339346
Epoch 4/30, Loss: 0.1826060491172891
Epoch 5/30, Loss: 0.17295636103341455
Epoch 6/30, Loss: 0.17251491154495038
Epoch 7/30, Loss: 0.1646222642377803
Epoch 8/30, Loss: 0.15898365629346747
Epoch 9/30, Loss: 0.15337324911042263
Epoch 10/30, Loss: 0.1522122110975416
Epoch 11/30, Loss: 0.14168761145127448
Epoch 12/30, Loss: 0.139103681162784
Epoch 13/30, Loss: 0.13911132412521462
Epoch 14/30, Loss: 0.1281212761213905
Epoch 15/30, Loss: 0.12801117567639603
Epoch 16/30, Loss: 0.1246933693948545
Epoch 17/30, Loss: 0.11523377385578658
Epoch 18/30, Loss: 0.10865355039897719
Epoch 19/30, Loss: 0.11958260340125937
Epoch 20/30, Loss: 0.10045395540563684
Epoch 21/30, Loss: 0.0991326284251715
Epoch 22/30, Loss: 0.09623174875190384
Epoch 23/30, Loss: 0.08864146976878769
Epoch 24/30, Loss: 0.08528193082464369
Epoch 25/30, Loss: 0.0857880573523672
Epoch 26/30, 

In [74]:
# Save models for comparison
torch.save(model_without_al.state_dict(), 'model_without_al.pth')
torch.save(model_with_al.state_dict(), 'model_with_al.pth')